In [1]:
import SimpleITK as sitk
import json
import numpy as np

# Must get csv and this jkson and other json for new datasets

In [3]:
template = "/work/vig/Datasets/aneurysm/hospital/crop_0.4_label/CA_000{}_0000.nii.gz"

dict_0_is_lower = {i: True for i in range(0, 38)}


mask_size_dict = {}
for num in range(0, 38):
    # zeropad num with fill
    num = str(num).zfill(2)
    print(num)
    mask_header = sitk.ReadImage(template.format(num))
    # get dimensions
    mask_size = mask_header.GetSize()
    mask_size_dict[f"CA_000{num}_0000.nii.gz"] = mask_size

00
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [4]:
# write as json

with open(
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/sizes/hospital.json", "w"
) as f:
    json.dump(mask_size_dict, f)

In [12]:
template = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4/Tr{}.nii.gz"

dict_0_is_lower = {i: True for i in range(1, 1187)}


mask_size_dict = {}
for num in range(1, 1187):
    # zeropad num with fill
    num = str(num).zfill(4)
    print(num)
    mask_header = sitk.ReadImage(template.format(num))
    # get dimensions
    mask_size = mask_header.GetSize()
    mask_size_dict[f"Tr{num}.nii.gz"] = mask_size


0001


0002
0003
0004
0005
0006
0007
0008
0009
0010
0011
0012
0013
0014
0015
0016
0017
0018
0019
0020
0021
0022
0023
0024
0025
0026
0027
0028
0029
0030
0031
0032
0033
0034
0035
0036
0037
0038
0039
0040
0041
0042
0043
0044
0045
0046
0047
0048
0049
0050
0051
0052
0053
0054
0055
0056
0057
0058
0059
0060
0061
0062
0063
0064
0065
0066
0067
0068
0069
0070
0071
0072
0073
0074
0075
0076
0077
0078
0079
0080
0081
0082
0083
0084
0085
0086
0087
0088
0089
0090
0091
0092
0093
0094
0095
0096
0097
0098
0099
0100
0101
0102
0103
0104
0105
0106
0107
0108
0109
0110
0111
0112
0113
0114
0115
0116
0117
0118
0119
0120
0121
0122
0123
0124
0125
0126
0127
0128
0129
0130
0131
0132
0133
0134
0135
0136
0137
0138
0139
0140
0141
0142
0143
0144
0145
0146
0147
0148
0149
0150
0151
0152
0153
0154
0155
0156
0157
0158
0159
0160
0161
0162
0163
0164
0165
0166
0167
0168
0169
0170
0171
0172
0173
0174
0175
0176
0177
0178
0179
0180
0181
0182
0183
0184
0185
0186
0187
0188
0189
0190
0191
0192
0193
0194
0195
0196
0197
0198
0199
0200
0201


 copy the header data
 set the array (updated)
 update the json files with the labels 
 code is somewhat messy, 
 training labels in the input have the size of the box in world coordinates, 
 but the center is in pixel index

if we update the offset we need to modify the csv files as well -training-
for froc evaluation we should do it in the world coordinates, the output 
must be converted before doing evaluation 


1. Scan: 
    Need to copy the header.
    Modify the volume.
    Change the origin, no other metadata should change.
    Can try to visualize the scan and the GT annotation.

2. Training CSV /work/vig/Datasets/aneurysm/test0.4.csv train0.4.csv <-
    Pixel: coordX,coordY,coordZ, <- only need to change this
    World: w,h,d
3. Metadata JSON files <- this one is easy if the first step is correct, we can just run the script to re-generate it and that's enough

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x7f89145717e0> >

In [ ]:
# write as json

with open(
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/scan_sizes_train_crop.json", "w"
) as f:
    json.dump(mask_size_dict, f)

In [8]:
path_file = "/work/vig/Datasets/aneurysm/internal_train/crop_0.4/Tr0001.nii.gz"


In [19]:
path_file = "/home/ceballosarroyo.a/workspace/datasets/aneurysm/external_test/ExtA0001.nii.gz"
mask_header = sitk.ReadImage(path_file)
mask_header.GetSpacing(), mask_header.GetSize()

((0.48828125, 0.48828125, 0.44999998807907104), (586, 586, 529))

In [20]:
def respace_to_0_4(path_file):
    image = sitk.ReadImage(path_file)
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator = sitk.sitkLinear
    resample.SetOutputDirection = image.GetDirection()
    resample.SetOutputOrigin = image.GetOrigin()
    new_spacing = np.array((0.4000000059604645, 0.4000000059604645, 0.4000000059604645))
    resample.SetOutputSpacing(new_spacing)
    orig_size = np.array(image.GetSize(), dtype=int)
    orig_spacing = image.GetSpacing()
    new_size = orig_size*(orig_spacing/new_spacing)
    new_size = np.ceil(new_size).astype(int) #  Image dimensions are in integers
    new_size = [int(s) for s in new_size]
    resample.SetSize(new_size)
    newimage = resample.Execute(image)
    return newimage

In [ ]:
respaced_image = respace_to_0_4(path_file)


((716, 716, 596), (0.4000000059604645, 0.4000000059604645, 0.4000000059604645))